In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import sys
import pyabc
import healpy as hp
from astropy.coordinates import SkyCoord
import astropy.units as u

sys.path.insert(0, '/home/aew492/lss-dipoles/code')
import tools
import abc_for_qso
import generate_mocks as gm

In [3]:
import cProfile
import pstats

#### log!

Originally $\rightarrow$ 1.75 sec per model + distance call!

Based on 10 model + distance calls (so taking total time / 10):
- After changing excess_map computation (replaced `sph_harm` functions with `hp.synfast()`) $\rightarrow$ 0.0204 sec per call
- After changing `sum` to `np.sum` in `distance()` $\rightarrow$ 0.0168 sec per call
- After changing `cmb_dipole` computation (moving `SkyCoord` outside of the model) $\rightarrow$ 0.0162 sec per call
- After stripping down `dipole_map()` to just `dipole()` (moving `theta, phi` calc. to outside `model()`) $\rightarrow$ 0.0129 sec per call

In [4]:
1.75 / .0129

135.65891472868216

#### set up test case

In [5]:
nside = 64
x0 = dict(data=np.ones(hp.nside2npix(nside)))

In [6]:
# selfunc
selfunc_str = 'quaia_G20.0_orig'
selfunc = gm.get_selfunc_map(selfunc_str)

# random parameters
pars = dict(dipole_amp=0.0052, log_excess=-5)

# base rate
base_rate = 33.6330

# cmb dipole dir
cmb_dipdir = SkyCoord(264, 48, unit=u.deg, frame='galactic')

# (theta, phi) in each healpixel
theta, phi = hp.pix2ang(nside, ipix=np.arange(hp.nside2npix(nside)))

In [7]:
def mock_and_distance(n):

    for i in range(n):
        mock = abc_for_qso.model(pars, selfunc, base_rate, cmb_dipdir, theta, phi)
        distance = abc_for_qso.distance(mock, x0, nside)

In [14]:
cProfile.run('mock_and_distance(10)', 'foo')
stats = pstats.Stats('foo')

In [15]:
stats.sort_stats('tottime').print_stats()

Mon Jan 13 21:47:32 2025    foo

         72824 function calls (71324 primitive calls) in 0.129 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       10    0.020    0.002    0.020    0.002 {method 'poisson' of 'numpy.random._generator.Generator' objects}
       10    0.008    0.001    0.008    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:761(ring2nest)
       10    0.007    0.001    0.007    0.001 {built-in method healpy._healpy_sph_transform_lib._alm2map}
       10    0.006    0.001    0.006    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:796(nest2ring)
       10    0.006    0.001    0.006    0.001 /home/aew492/lss-dipoles/code/dipole.py:11(dipole)
       20    0.004    0.000    0.020    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:831(reorder)
       10    0.003    0.000    0.103    0.010 /home/aew492/lss-dipoles/code/abc_for_qso.py:163(mode

In [30]:
stats.sort_stats('tottime').print_stats()

Thu Jan  9 22:18:19 2025    foo

         21877 function calls (21427 primitive calls) in 0.061 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.011    0.004    0.011    0.004 {built-in method builtins.sum}
        3    0.008    0.003    0.008    0.003 {method 'poisson' of 'numpy.random._generator.Generator' objects}
        3    0.004    0.001    0.004    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:485(pix2ang)
        3    0.003    0.001    0.003    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:761(ring2nest)
        3    0.002    0.001    0.002    0.001 {built-in method healpy._healpy_sph_transform_lib._alm2map}
        3    0.002    0.001    0.002    0.001 /home/aew492/lss-dipoles/code/dipole.py:11(dipole)
        3    0.002    0.001    0.002    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:796(nest2ring)
        6    0.002    

In [20]:
stats.sort_stats('tottime').print_stats()

Thu Jan  9 22:14:25 2025    foo

         24466 function calls (24016 primitive calls) in 0.072 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.013    0.004    0.013    0.004 {built-in method builtins.sum}
        3    0.006    0.002    0.006    0.002 {method 'poisson' of 'numpy.random._generator.Generator' objects}
        3    0.004    0.001    0.004    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:485(pix2ang)
        3    0.003    0.001    0.003    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:761(ring2nest)
        3    0.002    0.001    0.002    0.001 /home/aew492/lss-dipoles/code/dipole.py:11(dipole)
        6    0.002    0.000    0.008    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:831(reorder)
        3    0.002    0.001    0.002    0.001 {built-in method healpy._healpy_sph_transform_lib._alm2map}
        3    0.002    0.

In [39]:
def mock_and_distance():

    mock = abc_for_qso.model(pars, selfunc, base_rate)

    distance = abc_for_qso.distance(mock, x0, nside)

In [40]:
cProfile.run('mock_and_distance()', 'foo')
stats = pstats.Stats('foo')

In [41]:
stats.sort_stats('tottime').print_stats()

Thu Jan  9 18:00:49 2025    foo

         8158 function calls (8008 primitive calls) in 0.019 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.004    0.004    0.004    0.004 {built-in method builtins.sum}
        1    0.002    0.002    0.002    0.002 {method 'poisson' of 'numpy.random._generator.Generator' objects}
        1    0.001    0.001    0.001    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:485(pix2ang)
        1    0.001    0.001    0.001    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:761(ring2nest)
        1    0.001    0.001    0.001    0.001 {built-in method healpy._healpy_sph_transform_lib._alm2map}
        1    0.001    0.001    0.001    0.001 /ext3/miniconda3/lib/python3.10/site-packages/healpy/pixelfunc.py:796(nest2ring)
        1    0.001    0.001    0.001    0.001 /home/aew492/lss-dipoles/code/dipole.py:11(dipole)
        2    0.000    0.

In [ ]:
# 1.75 seconds originally per model + distance call!